# Document Re-Ranking with LLM-Based Relevance Scoring

In this section, I implement a **document re-ranking pipeline** using a language model (LLM) to assign **semantic relevance scores** to documents retrieved for a given query.

While traditional vector similarity search provides a fast way to find related content, it doesn’t always reflect **semantic precision** or **intent alignment**. To improve this, we prompt a powerful LLM to act as a relevance assessor.

In [1]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "LLM_Ranking"

# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

Environment Variables Set:
LANGSMITH_TRACING: true
LANGSMITH_ENDPOINT: https://api.smith.langchain.com
LANGSMITH_PROJECT: LLM_Reranking


In [3]:
import langchain
import sys
import os
import re
import pandas as pd
from tqdm import tqdm
from langchain.docstore.document import Document
from langchain_core.retrievers import BaseRetriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langsmith import traceable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from enum import Enum
from typing import Any, Dict, List, Tuple
from pydantic import BaseModel, Field

In [4]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


## LLM and Embedding Model

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=1000,
)

In [6]:
# Load the index back

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
new_vector_store = FAISS.load_local(
    "nvidia_large_baseline", 
    embeddings,  # Use the same embeddings used during creation
    allow_dangerous_deserialization=True  # Important for loading
)


retriever = new_vector_store.as_retriever()

## LLM reranker 

In [7]:
class RatingScore(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of a document to a query.")


def rerank_documents(query: str, docs: List[Document], top_n: int = 4) -> List[Tuple[Document, float]]:
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )

    # llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)
    llm_chain = prompt_template | llm.with_structured_output(RatingScore)

    scored_docs = []
    
    # Show progress bar while processing documents
    for doc, score in tqdm(docs, desc="Scoring Documents", unit="doc"):
        input_data = {"query": query, "doc": doc.page_content}
        try:
            score = llm_chain.invoke(input_data).relevance_score
            score = float(score)  # Ensure float conversion
        except (ValueError, AttributeError):
            score = 0  # Default score if parsing fails
        scored_docs.append((doc, score))

    # Sorting based on score in descending order
    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    
    return reranked_docs[:top_n]  # Ensure it returns (doc, score) pairs


## Small Experiment

In [8]:
question = "Who is the CEO of NVIDIA?"
retrieved_docs_with_scores = retriever.vectorstore.similarity_search_with_score(question, k=15)

In [9]:
def create_document_dataframe(docs):
    """
    Creates a DataFrame from retrieved document details and scores.
    """
    return pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Similarity Score": round(score, 4),  # Rounding for readability
            "Retrieved Chunk": doc.page_content[:1000] + "..."  # Show up to 1000 characters
        }
        for doc, score in docs
    ])

In [10]:
def display_document_dataframe(df, name="Document Details"):
    """
    Configures DataFrame display settings and displays it.
    """
    pd.set_option("display.max_colwidth", None)  # Do not truncate column content
    pd.set_option("display.width", 1000)  # Set a larger display width
    
    # Display the DataFrame
    display(df)

In [11]:
df = create_document_dataframe(retrieved_docs_with_scores)
display_document_dataframe(df)

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,0.5491,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,11,0.6880,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
2,NVIDIA_Report_2024.pdf,12,0.8722,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
3,NVIDIA_Report_2024.pdf,12,0.9087,"Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinnesota, an M.S.E.E. degree from the California Institute of Technology and an M.B.A. degree from Harvard Business School.\nDebora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement..."
4,NVIDIA_Report_2024.pdf,33,0.9121,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
5,NVIDIA_Report_2024.pdf,

In [12]:
reranked_docs = rerank_documents(question, retrieved_docs_with_scores)

Scoring Documents: 100%|███████████████████████| 15/15 [00:11<00:00,  1.25doc/s]


In [13]:
df2 = create_document_dataframe(reranked_docs)
display_document_dataframe(df2)

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,89,10.0,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and..."
1,NVIDIA_Report_2024.pdf,11,9.0,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
2,NVIDIA_Report_2024.pdf,11,9.0,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,84,9.0,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024..."


## Create a custom retriever based on our reranker

In [14]:
class CustomRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Vector store for initial retrieval")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str, num_docs: int = 4) -> List[Document]:
        try:
            initial_docs = self.vectorstore.similarity_search(query, k=30)
            return rerank_documents(query, initial_docs, top_n=num_docs)
        except Exception as e:
            print(f"Retrieval error: {e}")
            return []


/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_4093/2717287172.py:1: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CustomRetriever(BaseRetriever, BaseModel):


In [15]:
# Create the custom retriever
custom_retriever = CustomRetriever(vectorstore=new_vector_store)

## But we don't use a class in the sake of simpler display

In [16]:
def setup_rag_pipeline():
    template = """"
    You are a helpful assistant that answers questions based on the provided context.
    Use the provided context to answer the question.
    Question: {input}
    Context: {context}
    Answer:
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    # Setup RAG pipeline
    return (
        {"context": RunnablePassthrough(), "input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# Initialize the RAG chain with tracing
rag_chain = setup_rag_pipeline()


In [17]:
def query_rag(question: str, top_k=15):
    """
    Query the RAG pipeline, retrieve relevant documents, and display their similarity scores in descending order.
    Returns both the generated answer and retrieved contexts.
    """
    # Retrieve the top-k chunks with similarity scores
    retrieved_docs_with_scores = retriever.vectorstore.similarity_search_with_score(question, k=top_k)
    
    # Display initial
    
    data_frame_initial = create_document_dataframe(retrieved_docs_with_scores)
    
    display_document_dataframe(data_frame_initial)
    
    reranked_docs = rerank_documents(question, retrieved_docs_with_scores)
    
    # Display reranked
    
    data_frame_reranked = create_document_dataframe(reranked_docs)
    
    display_document_dataframe(data_frame_reranked)

    # Extract contexts (raw document texts)
    retrieved_contexts = [doc.page_content for doc, _ in reranked_docs]

    # Run the RAG pipeline with the retrieved context
#     answer = rag_chain.invoke(question)
    
    answer = rag_chain.invoke({"input": question, "context": reranked_docs})

    # Return both the answer and the retrieved contexts
    return {
        "answer": answer,
        "contexts": retrieved_contexts
    }

In [18]:
response = query_rag("When did the CEO join the firm?")

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,1.2053,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,48,1.2284,"/s/ PricewaterhouseCoopers LLP\nSan Jose, CaliforniaFebruary 21, 2024\nWe have served as the Company’s auditor since 2004.\n49..."
2,NVIDIA_Report_2024.pdf,12,1.3033,"Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinnesota, an M.S.E.E. degree from the California Institute of Technology and an M.B.A. degree from Harvard Business School.\nDebora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement..."
3,NVIDIA_Report_2024.pdf,11,1.3175,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
4,NVIDIA_Report_2024.pdf,12,1.3268,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
5,NVIDIA_Report_2024.pdf,84,1.3594,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS

Scoring Documents: 100%|███████████████████████| 15/15 [00:16<00:00,  1.08s/doc]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,8.0,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,84,8.0,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024..."
2,NVIDIA_Report_2024.pdf,11,3.0,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,12,3.0,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."


In [19]:
response["answer"]

"Jen-Hsun Huang co-founded NVIDIA in 1993 and has served as the President and Chief Executive Officer since the company's inception."

In [20]:
len(response["contexts"])

4

## RAG Dataset

In [21]:
# Import the LangSmith client
from langsmith import Client

# Initialize the LangSmith client
ls_client = Client()

# Read an existing dataset by name
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples
for example in examples:
    print(example.inputs)  # Print input data
    print(example.outputs)  # Print output data


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

In [22]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_4093/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


## RAG Evaluators

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [23]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag(example["question"])
    return {"answer": response["answer"]}

In [24]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

### RAG Answer VS Reference prompt

In [25]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [26]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model
 
    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [75]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="LLM_Reranker",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'LLM_Reranker-3ed931fb' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=9c52c9e5-3ff6-40f7-856c-3fd982882cfe




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,0.5491,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,11,0.6880,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
2,NVIDIA_Report_2024.pdf,12,0.8722,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
3,NVIDIA_Report_2024.pdf,12,0.9087,"Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinnesota, an M.S.E.E. degree from the California Institute of Technology and an M.B.A. degree from Harvard Business School.\nDebora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement..."
4,NVIDIA_Report_2024.pdf,33,0.9121,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
5,NVIDIA_Report_2024.pdf,


Scoring Documents:   0%|                                | 0/15 [00:00<?, ?doc/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,0.5249,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,3,0.6685,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,4,0.6938,"Professional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
3,NVIDIA_Report_2024.pdf,6,0.7180,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
4,NVIDIA_Report_2024.pdf,4,0.7326,"The NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific\ncomputing, delivering significantly better performance and power efficiency relative to conventional CPU-only approaches. It is deployed in cloud, hyperscale,on-premises and edge data centers. The platform consists of compute and networking offerings typically delivered to customers as systems, subsystems, ormodules, along with software and services...."
5,NVIDIA_Report_2024.pdf,33,0.7348,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, r

...

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,89,10.0,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and..."
1,NVIDIA_Report_2024.pdf,11,9.0,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
2,NVIDIA_Report_2024.pdf,11,9.0,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,84,9.0,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024..."





Scoring Documents:  87%|███████████████████▉   | 13/15 [00:06<00:01,  1.72doc/s]

Scoring Documents:  93%|█████████████████████▍ | 14/15 [00:07<00:00,  1.94doc/s]


Scoring Documents:  93%|█████████████████████▍ | 14/15 [00:07<00:00,  1.81doc/s]

Scoring Documents: 100%|███████████████████████| 15/15 [00:08<00:00,  1.79doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,9.0,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,6,8.0,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
2,NVIDIA_Report_2024.pdf,33,8.0,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
3,NVIDIA_Report_2024.pdf,4,8.0,"Our Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for..."





Scoring Documents: 100%|███████████████████████| 15/15 [00:07<00:00,  1.90doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,49,9.0,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
1,NVIDIA_Report_2024.pdf,35,9.0,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
2,NVIDIA_Report_2024.pdf,78,9.0,"Sales to one customer represented 13% of total revenue for fiscal year 2024, which was attributable to the Compute & Networking segment. No customerrepresented 10% or more of total revenue for fiscal years 2023 and 2022.\nThe following table summarizes information pertaining to our revenue by each of the specialized markets we serve:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nData Center $ 47,525 $ 15,005 $ 10,613 \nGaming 10,447 9,067 12,462 \nProfessional Visualization 1,553 1,544 2,111 \nAutomotive 1,091 903 566 \nOEM and Other 306 455 1,162 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \n79..."
3,NVIDIA_Report_2024.pdf,38,9.0,"Income tax expense (benefit) 6.6 (0.7)\nNet income 48.9 % 16.2 %\nReportable Segments\nRevenue by Reportable Segments\nYear Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 47,405 $ 15,068 $ 32,337 215 %\nGraphics 13,517 11,906 1,611 14 %\nTotal $ 60,922 $ 26,974 $ 33,948 126 %\nOperating Income by Reportable Segments\nYear Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 32,016 $ 5,083 $ 26,933 530 %\nGraphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %..."


### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [31]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [32]:
# Prompt 

# grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

In [78]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="LLM_Reranker",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'LLM_Reranker-928ebd6d' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=57b2cd5e-59d5-447e-9a50-fcda8b58686c




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,0.5505,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
1,NVIDIA_Report_2024.pdf,35,0.6613,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,49,0.6642,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
3,NVIDIA_Report_2024.pdf,35,0.7459,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
4,NVIDIA_Report_2024.pdf,35,0.7577,"growing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory provisions as a percentage of revenue.\nOperating expenses increased for fiscal year 2024, driven by growth in employees and compensation increases. Fiscal year 2023 also included a $1.4 billion\nacquisition termination charge related to the proposed Arm transaction.\nMarket Platform Highlights\nData Center revenue for fiscal year 2024 wa


Scoring Documents:   0%|                                | 0/15 [00:00<?, ?doc/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,0.5249,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,3,0.6685,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,4,0.6938,"Professional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
3,NVIDIA_Report_2024.pdf,6,0.7180,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
4,NVIDIA_Report_2024.pdf,4,0.7326,"The NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific\ncomputing, delivering significantly better performance and power efficiency relative to conventional CPU-only approaches. It is deployed in cloud, hyperscale,on-premises and edge data centers. The platform consists of compute and networking offerings typically delivered to customers as systems, subsystems, ormodules, along with software and services...."
5,NVIDIA_Report_2024.pdf,33,0.7348,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, r

...

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,49,9.0,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
1,NVIDIA_Report_2024.pdf,35,9.0,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
2,NVIDIA_Report_2024.pdf,78,9.0,"Sales to one customer represented 13% of total revenue for fiscal year 2024, which was attributable to the Compute & Networking segment. No customerrepresented 10% or more of total revenue for fiscal years 2023 and 2022.\nThe following table summarizes information pertaining to our revenue by each of the specialized markets we serve:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nData Center $ 47,525 $ 15,005 $ 10,613 \nGaming 10,447 9,067 12,462 \nProfessional Visualization 1,553 1,544 2,111 \nAutomotive 1,091 903 566 \nOEM and Other 306 455 1,162 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \n79..."
3,NVIDIA_Report_2024.pdf,38,9.0,"Income tax expense (benefit) 6.6 (0.7)\nNet income 48.9 % 16.2 %\nReportable Segments\nRevenue by Reportable Segments\nYear Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 47,405 $ 15,068 $ 32,337 215 %\nGraphics 13,517 11,906 1,611 14 %\nTotal $ 60,922 $ 26,974 $ 33,948 126 %\nOperating Income by Reportable Segments\nYear Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 32,016 $ 5,083 $ 26,933 530 %\nGraphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %..."




Scoring Documents:  93%|█████████████████████▍ | 14/15 [00:08<00:00,  1.87doc/s]


Scoring Documents: 100%|███████████████████████| 15/15 [00:08<00:00,  1.82doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,89,10.0,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and..."
1,NVIDIA_Report_2024.pdf,11,9.0,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
2,NVIDIA_Report_2024.pdf,11,9.0,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,84,9.0,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024..."




Scoring Documents: 100%|███████████████████████| 15/15 [00:08<00:00,  1.76doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,9.0,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,6,8.0,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
2,NVIDIA_Report_2024.pdf,33,8.0,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
3,NVIDIA_Report_2024.pdf,4,8.0,"Our Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for..."


### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [28]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [29]:
# Grade prompt 
# grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [81]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[docs_relevance_evaluator],
    experiment_prefix="LLM_Reranker",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'LLM_Reranker-c3f03798' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=a8c499d1-1169-4f5e-91e6-303e98311354




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,0.5506,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
1,NVIDIA_Report_2024.pdf,35,0.6613,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,49,0.6642,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
3,NVIDIA_Report_2024.pdf,35,0.7459,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
4,NVIDIA_Report_2024.pdf,35,0.7577,"growing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory provisions as a percentage of revenue.\nOperating expenses increased for fiscal year 2024, driven by growth in employees and compensation increases. Fiscal year 2023 also included a $1.4 billion\nacquisition termination charge related to the proposed Arm transaction.\nMarket Platform Highlights\nData Center revenue for fiscal year 2024 wa


Scoring Documents:   0%|                                | 0/15 [00:00<?, ?doc/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,0.5249,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,3,0.6685,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,4,0.6939,"Professional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
3,NVIDIA_Report_2024.pdf,6,0.7180,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
4,NVIDIA_Report_2024.pdf,4,0.7326,"The NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific\ncomputing, delivering significantly better performance and power efficiency relative to conventional CPU-only approaches. It is deployed in cloud, hyperscale,on-premises and edge data centers. The platform consists of compute and networking offerings typically delivered to customers as systems, subsystems, ormodules, along with software and services...."
5,NVIDIA_Report_2024.pdf,33,0.7349,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, r

...

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,89,10.0,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and..."
1,NVIDIA_Report_2024.pdf,11,9.0,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
2,NVIDIA_Report_2024.pdf,11,9.0,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,84,9.0,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024..."




Scoring Documents: 100%|███████████████████████| 15/15 [00:07<00:00,  1.89doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,9.0,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,6,8.0,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
2,NVIDIA_Report_2024.pdf,33,8.0,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
3,NVIDIA_Report_2024.pdf,4,8.0,"Our Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for..."



Scoring Documents: 100%|███████████████████████| 15/15 [00:08<00:00,  1.70doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,49,9.0,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
1,NVIDIA_Report_2024.pdf,35,9.0,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
2,NVIDIA_Report_2024.pdf,78,9.0,"Sales to one customer represented 13% of total revenue for fiscal year 2024, which was attributable to the Compute & Networking segment. No customerrepresented 10% or more of total revenue for fiscal years 2023 and 2022.\nThe following table summarizes information pertaining to our revenue by each of the specialized markets we serve:\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue: (In millions)\nData Center $ 47,525 $ 15,005 $ 10,613 \nGaming 10,447 9,067 12,462 \nProfessional Visualization 1,553 1,544 2,111 \nAutomotive 1,091 903 566 \nOEM and Other 306 455 1,162 \nTotal revenue $ 60,922 $ 26,974 $ 26,914 \n79..."
3,NVIDIA_Report_2024.pdf,38,9.0,"Income tax expense (benefit) 6.6 (0.7)\nNet income 48.9 % 16.2 %\nReportable Segments\nRevenue by Reportable Segments\nYear Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 47,405 $ 15,068 $ 32,337 215 %\nGraphics 13,517 11,906 1,611 14 %\nTotal $ 60,922 $ 26,974 $ 33,948 126 %\nOperating Income by Reportable Segments\nYear Ended\nJan 28, 2024 Jan 29, 2023 $Change %Change\n($ in millions)\nCompute & Networking $ 32,016 $ 5,083 $ 26,933 530 %\nGraphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %..."


## Complete Evaluation

In [33]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="LLM_Reranker",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'LLM_Reranker-1c73d7b1' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=0e34414f-e313-4d9f-baff-c15b4e62d200




0it [00:00, ?it/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,0.8052,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,11,0.8149,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
2,NVIDIA_Report_2024.pdf,59,0.8422,"In February 2022, NVIDIA and SoftBank Group Corp, or SoftBank, announced the termination of the Share Purchase Agreement whereby NVIDIA would have\nacquired Arm from SoftBank. The parties agreed to terminate it due to significant regulatory challenges preventing the completion of the transaction. Werecorded an acquisition termination cost of $1.4 billion in fiscal year 2023 reflecting the write-off of the prepayment provided at signing.\nNote 3 - Leases\nOur lease obligations primarily consist of operating leases for our headquarters complex, domestic and international office facilities, and data center space, withlease periods expiring between fiscal years 2025 and 2035.\nFuture minimum lease payments under our non-cancelable operating leases as of January 28, 2024, are as follows:..."
3,NVIDIA_Report_2024.pdf,12,0.8860,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
4,NVIDIA_Report_2024.pdf,12,0.8966,"Puri previously held marketing, management consulting, and product development positions at Hewlett-Packard, an information technology company, Booz AllenHamilton Inc., a management and technology consulting company, and Texas Instruments Incorporated. Mr. Puri holds a B.S.E.E. degree from the University ofMinnesota, an M.S.E.E. degree from the California Institute of Technology and an M.B.A. degree from Harvard Business School.\nDebora Shoquist joined NVIDIA in 2007 as Senior Vice President of Operations and in 2009 became Executive Vice President of Operations. Prior to NVIDIA,Ms. Shoquist served from 2004 to 2007 as Executive Vice President of Operations at JDS Uniphase Corp., a provider of communications test and measurement..."
5,NVIDIA_Report_2024


Scoring Documents:   0%|                                | 0/15 [00:00<?, ?doc/s]

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,39,0.7911,"fabrication, assembly, testing and packaging, board and device costs, manufacturing support costs, including labor and overhead associated with suchpurchases, final test yield fallout, inventory and warranty provisions, memory and component costs, tariffs, and shipping costs. Cost of revenue also includesacquisition-related costs, development costs for license and service arrangements, IP-related costs, and stock-based compensation related to personnel\nassociated with manufacturing operations.\nOur overall gross margin increased to 72.7% in fiscal year 2024 from 56.9% in fiscal year 2023. The year over year increase was primarily due to strong Data\nCenter revenue growth of 217% and lower net inventory provisions as a percentage of revenue...."
1,NVIDIA_Report_2024.pdf,38,0.8171,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."
2,NVIDIA_Report_2024.pdf,35,0.8181,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
3,NVIDIA_Report_2024.pdf,49,0.8384,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
4,NVIDIA_Report_2024.pdf,78,0.8720,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements(Continued)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\n(In millions)\nReconciling items included in ""All Other"" category:\nStock-based compensation expense $ (3,549)$ (2,710)$ (2,004)\nUnallocated cost of revenue and operating expenses (728) (595) (399)\nAcquisition-related and other costs (583) (674) (636)\nIP-related and legal settlement costs (40) (23) (10)\nRestructuring costs and other — (54) — \nAcquisition termination cost — (1,353) — \nOther 10 (2) — \nTotal $ (4,890)$ (5,411) $ (3,049)\nRevenue by geographic areas is desi

...

,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,89,10.0,"EXHIBIT 32.1\n \nCERTIFICATION \n \nPursuant to the requirement set forth in Rule 13a-14(b) of the Securities Exchange Act of 1934, as amended (the “Exchange Act”), and\nSection 1350 of Chapter 63 of Title 18 of the United States Code (18 U.S.C. § 1350), Jen-Hsun Huang, the President and Chief Executive\nOfficer of NVIDIA Corporation (the “Company”), hereby certifies that, to the best of his knowledge:\n1. The Company's Annual Report on Form 10-K for the year ended January 28, 2024, to which this Certification is attached as Exhibit 32.1 (the\n“Periodic Report”), fully complies with the requirements of Section 13(a) or Section 15(d) of the Exchange Act; and..."
1,NVIDIA_Report_2024.pdf,11,9.0,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
2,NVIDIA_Report_2024.pdf,11,9.0,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,84,9.0,"Pursuant to the requirements of the Securities Exchange Act of 1934, this report has been signed below by the following persons on behalf of the registrant andin the capacities and on the dates indicated.\nSignature Title Date\n/s/ JEN-HSUN HUANG President, Chief Executive Officer and Director(Principal Executive Officer) February 21, 2024\nJen-Hsun Huang \n/s/ COLETTE M. KRESS Executive Vice President and Chief Financial Officer(Principal Financial Officer) February 21, 2024\nColette M. Kress \n/s/ DONALD ROBERTSON Vice President and Chief Accounting Officer(Principal Accounting Officer) February 21, 2024\nDonald Robertson\n/s/ ROBERT BURGESS Director February 21, 2024\nRobert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024..."









Scoring Documents: 100%|███████████████████████| 15/15 [00:10<00:00,  1.50doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,9.0,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,6,8.0,"layers. This full-stack innovation approach allows us to deliver order-of-magnitude performance advantages relative to legacy approaches in our target markets,which include Data Center, Gaming, Professional Visualization, and Automotive. While the computing requirements of these end markets are diverse, weaddress them with a unified underlying architecture leveraging our GPUs, CUDA and networking technologies as the fundamental building blocks. The\nprogrammable nature of our architecture allows us to make leveraged investments in research and development: we can support several multi-billion-dollar endmarkets with shared underlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. We utilize..."
2,NVIDIA_Report_2024.pdf,33,8.0,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
3,NVIDIA_Report_2024.pdf,4,8.0,"Our Businesses\nWe report our business results in two segments.\nThe Compute & Networking segment is comprised of our Data Center accelerated computing platforms and end-to-end networking platforms including Quantumfor InfiniBand and Spectrum for Ethernet; our NVIDIA DRIVE automated-driving platform and automotive development agreements; Jetson robotics and other\nembedded platforms; NVIDIA AI Enterprise and other software; and DGX Cloud software and services.\nThe Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure; Quadro/NVIDIARTX GPUs for enterprise workstation graphics; virtual GPU, or vGPU, software for cloud-based visual and virtual computing; automotive platforms for..."






Scoring Documents:  80%|██████████████████▍    | 12/15 [00:12<00:06,  2.20s/doc]



Scoring Documents:  87%|███████████████████▉   | 13/15 [00:13<00:03,  1.74s/doc]



Scoring Documents:  93%|█████████████████████▍ | 14/15 [00:14<00:01,  1.48s/doc]



Scoring Documents: 100%|███████████████████████| 15/15 [00:14<00:00,  1.02doc/s]


,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,25,8.0,"on our ability to conduct our business, any of which could have a material and adverse impact on our business, operating results and financial condition.\nDuring the third quarter of fiscal year 2023, the USG announced export restrictions and export licensing requirements targeting China’s semiconductor and\nsupercomputing industries. These restrictions impact exports of certain chips, as well as software, hardware, equipment and technology used to develop,produce and manufacture certain chips to China (including Hong Kong and Macau) and Russia, and specifically impact our A100 and H100 integrated circuits,DGX or any other systems or boards which incorporate A100 or H100 integrated circuits. The licensing requirements also apply to any future NVIDIA integrated..."
1,NVIDIA_Report_2024.pdf,34,8.0,"Table of Contents\nGlobal Trade\nDuring the third quarter of fiscal year 2023, the USG, announced licensing requirements that, with certain exceptions, impact exports to China (including Hong\nKong and Macau) and Russia of our A100 and H100 integrated circuits, DGX or any other systems or boards which incorporate A100 or H100 integrated circuits.\nIn July 2023, the USG informed us of an additional licensing requirement for a subset of A100 and H100 products destined to certain customers and other\nregions, including some countries in the Middle East...."
2,NVIDIA_Report_2024.pdf,9,8.0,"In October 2023, the USG announced new and updated licensing requirements that became effective in our fourth quarter of fiscal year 2024 for exports to\nChina and Country Groups D1, D4, and D5 (including but not limited to Saudi Arabia, the United Arab Emirates, and Vietnam, but excluding Israel) of ourproducts exceeding certain performance thresholds, including A100, A800, H100, H800, L4, L40, L40S and RTX 4090. The licensing requirements also apply to\nthe export of products exceeding certain performance thresholds to a party headquartered in, or with an ultimate parent headquartered in, Country Group D5,including China. On October 23, 2023, the USG informed us the licensing requirements were effective immediately for shipments of our A100, A800, H100,H800, and L40S products...."
3,NVIDIA_Report_2024.pdf,34,8.0,"regions, including some countries in the Middle East.\nIn October 2023, the USG announced new and updated licensing requirements that became effective in our fourth quarter of fiscal year 2024 for exports toChina and Country Groups D1, D4, and D5 (including but not limited to Saudi Arabia, the United Arab Emirates, and Vietnam, but excluding Israel) of our\nproducts exceeding certain performance thresholds, including A100, A800, H100, H800, L4, L40, L40S and RTX 4090. The licensing requirements also apply tothe export of products exceeding certain performance thresholds to a party headquartered in, or with an ultimate parent headquartered in, Country Group D5,..."
